<a href="https://colab.research.google.com/github/halfnet/BacktestR/blob/master/BFA_lost_sales_prototype_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import pandas_gbq
import datetime
from google.cloud import bigquery
import pandas as pd
import numpy as np

In [3]:
def prototype_run(proposal_id):

  print("-----" + str(proposal_id) + "-----")
  client = bigquery.Client(project=project_id)  

  sql ="""\
    SELECT 
      snapshot_id,
      DATE(refresh.brstarttime) refresh_date, 
      order_cadence,
    FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal` proposal
    INNER JOIN `wf-gcp-us-ae-ops-prod.buyfair_bulk.tbl_buyfair_refresh_vertica` refresh
      ON proposal.snapshot_id = refresh.brid
    WHERE proposal_id = {}
  """
  query=sql.format(proposal_id)
  df = client.query(query).to_dataframe()
  snapshot_id = df.loc[0, "snapshot_id"]
  start_date = df.loc[0, "refresh_date"]
  order_cadence = df.loc[0, "order_cadence"]
  sql ="""\
    SELECT
      MAX(arrival_date) arrival_date
    FROM
      `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
    WHERE proposal_id = {}
  """
  query=sql.format(proposal_id)
  df = client.query(query).to_dataframe()
  end_date = df.loc[0, "arrival_date"]
  end_date = end_date + datetime.timedelta(int(order_cadence))
  ##### Delete Existing Data #####
  sql ="""\
      DELETE FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      WHERE proposal_id = {}
  """
  query=sql.format(proposal_id)
  df = client.query(query).to_dataframe()
  ##### Actual Case #####
  sql ="""\
      INSERT `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
        (
          analysis_case,
          proposal_id,
          supplier_id,
          manufacturer_part_id,
          sprid,
          date,
          is_oos,
          available_qty,
          receipt_qty,
          demand_qty,
          potential_lost_sales,
          wholesale_cost_per_unit,
          revenue_per_unit,
          expected_lost_sales,
          expected_lost_sales_by_cost,
          expected_lost_sales_by_revenue
        )
      WITH item_start_end_dates AS
      (
        SELECT supplier_id, manufacturer_part_id, sprid,
          CAST('{}' AS DATE) start_date,
          CAST('{}' AS DATE) end_date 
        FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal` 
        WHERE proposal_id = {}
        GROUP BY 1, 2, 3
      ),
      spo_receipt AS
      (
        SELECT manufacturer_part_id, receipt_date, SUM(receipt_qty) receipt_qty
        FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_spo_receipt`
        GROUP BY 1, 2
      )
      SELECT 
        'actual' analysis_case,
        {} proposal_id,
        dates.supplier_id,
        ls.manufacturer_part_id,
        dates.sprid,
        order_date date,
        is_oos,
        inv.pickable_qty,
        --IFNULL(sr.receipt_qty, 0) receipt_qty,
        0 receipt_qty,
        order_qty demand_qty,
        ls_cg.lost_sales potential_lost_sales,
        wholesale_cost_per_unit,
        revenue_per_unit,
        ls_cg.lost_sales * is_oos expected_lost_sales,
        ls_cg.lost_sales * is_oos * wholesale_cost_per_unit expected_lost_sales_by_cost,
        ls_cg.lost_sales * is_oos * revenue_per_unit expected_lost_sales_by_revenue
      FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales` ls
      INNER JOIN item_start_end_dates dates
        ON ls.manufacturer_part_id = dates.manufacturer_part_id 
        AND ls.order_date BETWEEN dates.start_date AND dates.end_date
      INNER JOIN `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_cg_only` ls_cg
        ON ls.manufacturer_part_id = ls_cg.manufacturer_part_id 
        AND ls.order_date = ls_cg.date
      LEFT OUTER JOIN `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_product_inventory_daily` inv
        ON ls.order_date = inv.effect_date AND ls.manufacturer_part_id = inv.manufacturer_part_id
      LEFT OUTER JOIN spo_receipt sr
        ON ls.order_date = sr.receipt_date AND ls.manufacturer_part_id = sr.manufacturer_part_id;
  """
  query=sql.format(start_date, end_date, proposal_id, proposal_id)
  df = client.query(query).to_dataframe()
  ##### Hypothetical Case #####
  # this below is ran after the full time range of impact has been passed
  sql ="""\
      WITH onhand as (
        SELECT
          supplier_id,
          proposal_id,
          sprid,
          manufacturer_part_id,
          MIN(on_hand_qty) AS on_hand,
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1,2,3,4),
      proposal0 as (
        SELECT
          l1.manufacturer_part_id,
          l1.arrival_date AS date,
          SUM(l1.quantity) AS quantity,
          AVG(l2.quantity) AS quantity2,
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal` l1
        LEFT OUTER JOIN `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_l2` l2
          ON l1.proposal_id = l2.proposal_id AND l1.manufacturer_part_id = l2.manufacturer_part_id
        WHERE l1.proposal_id = {}
        GROUP BY
          1,2
      ),
      proposal as (
        SELECT
          manufacturer_part_id,
          date,
          quantity,
          quantity2,
          SUM(quantity) OVER (
            PARTITION BY manufacturer_part_id
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS total_qty,
        FROM proposal0
      ),
      proposal_l1 as (
        SELECT
          manufacturer_part_id,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1
      ),
      proposal_l2 as (
        SELECT
          manufacturer_part_id,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_l2`
        WHERE proposal_id = {}
        GROUP BY
          1
      ),
      proposal_l4 as (
        SELECT
          manufacturer_part_id,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_l4`
        WHERE proposal_id = {}
        GROUP BY
          1
      ),
      proposal_receipt AS (
        SELECT manufacturer_part_id, 
        DATE_ADD(receipt_date, INTERVAL 2 DAY) AS date, 
        SUM(receipt_qty) receipt_qty
        FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_spo_receipt`
        WHERE spo IN 
          (SELECT DISTINCT spo FROM
            `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_spo`
            WHERE proposal_id = {})
        GROUP BY 1, 2
      ),
      other_receipt AS (
        SELECT manufacturer_part_id, 
        DATE_ADD(receipt_date, INTERVAL 2 DAY) AS date, 
        SUM(receipt_qty) receipt_qty
        FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_spo_receipt`
        WHERE spo NOT IN 
          (SELECT DISTINCT spo FROM
            `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_spo`
            WHERE proposal_id = {})
        GROUP BY 1, 2
      ),
      date_range as (
        SELECT * FROM UNNEST(GENERATE_DATE_ARRAY('{}', '{}', INTERVAL 1 DAY)) AS date
      )
      SELECT
        onhand.proposal_id,
        onhand.supplier_id,
        onhand.manufacturer_part_id,
        onhand.sprid,
        date_range.date,
        CASE WHEN date_range.date = '{}' THEN on_hand ELSE 0 END as start_qty,
        CASE WHEN date_range.date = '{}' THEN on_hand ELSE 0 END as start_qty2,
        IFNULL(other_receipt.receipt_qty, 0) AS other_receipt_qty,
        IFNULL(proposal_receipt.receipt_qty, 0) AS proposal_receipt_qty,
        IFNULL(proposal.quantity, 0) AS proposal_qty,
        IFNULL(1.0 * proposal.quantity2 * proposal.quantity / proposal.total_qty, 0) AS proposal_qty2,
        IFNULL(proposal_l1.quantity, 0) AS proposal_l1_qty,
        IFNULL(proposal_l2.quantity, 0) AS proposal_l2_qty,
        IFNULL(proposal_l4.quantity, 0) AS proposal_l4_qty,
        SUM(IFNULL(proposal_receipt.receipt_qty, 0))
        OVER (
          PARTITION BY onhand.manufacturer_part_id
          ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS total_proposal_receipts,
        IFNULL(ls.order_qty, 0) order_qty,
        IFNULL(ls.is_oos, 0) actual_is_oos,
        IFNULL(ls.wholesale_cost_per_unit, 0) wholesale_cost_per_unit,
        IFNULL(ls.revenue_per_unit, 0) revenue_per_unit,
        IFNULL(ls_cg.lost_sales, 0) lost_sales,
      FROM
        onhand
      CROSS JOIN
        date_range
      LEFT JOIN 
        proposal 
      ON
        proposal.manufacturer_part_id = onhand.manufacturer_part_id 
        AND proposal.date = date_range.date
      LEFT JOIN 
        proposal_l1
      ON
        proposal_l1.manufacturer_part_id = onhand.manufacturer_part_id
      LEFT JOIN 
        proposal_l2
      ON
        proposal_l2.manufacturer_part_id = onhand.manufacturer_part_id
      LEFT JOIN 
        proposal_l4
      ON
        proposal_l4.manufacturer_part_id = onhand.manufacturer_part_id 
      LEFT JOIN 
        proposal_receipt 
      ON
        proposal_receipt.manufacturer_part_id = onhand.manufacturer_part_id 
        AND proposal_receipt.date = date_range.date
      LEFT JOIN 
        other_receipt 
      ON
        other_receipt.manufacturer_part_id = onhand.manufacturer_part_id 
        AND other_receipt.date = date_range.date
      LEFT JOIN
        `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales` AS ls
      ON
        onhand.manufacturer_part_id = ls.manufacturer_part_id
        AND date_range.date = ls.order_date
      LEFT JOIN 
        `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_cg_only` ls_cg
      ON 
        date_range.date = ls_cg.date
        AND ls_cg.manufacturer_part_id = onhand.manufacturer_part_id
      ORDER BY
        onhand.manufacturer_part_id ASC,
        date_range.date ASC;
  """
  query=sql.format(proposal_id, proposal_id, proposal_id, proposal_id,
                  proposal_id, proposal_id, proposal_id,   
                  start_date, end_date, start_date, start_date)
  df = client.query(query).to_dataframe()
  df["hypothetical_qty"] = np.where((df.proposal_l4_qty == 0) | (df.total_proposal_receipts == 0), df.proposal_qty, 
                                    1.0 * df.proposal_l1_qty * df.proposal_receipt_qty / df.total_proposal_receipts)
  df["hypothetical_qty2"] = np.where((df.proposal_l4_qty == 0) | (df.total_proposal_receipts == 0), df.proposal_qty2, 
                                    1.0 * df.proposal_l2_qty * df.proposal_receipt_qty / df.total_proposal_receipts)
  df["start_qty"] = df["start_qty"].astype(float)
  df["start_qty2"] = df["start_qty2"].astype(float)
  df["other_receipt_qty"] = df["other_receipt_qty"].astype(float)
  df["order_qty"] = df["order_qty"].astype(float)
  df["wholesale_cost_per_unit"] = df["wholesale_cost_per_unit"].astype(float)
  df["revenue_per_unit"] = df["revenue_per_unit"].astype(float)
  df["date"] = pd.to_datetime(df["date"])
  df.drop(columns=["proposal_receipt_qty", "proposal_qty", "proposal_qty2", "proposal_l1_qty", "proposal_l2_qty",
                  "proposal_l4_qty", "total_proposal_receipts"], inplace=True)
  print (df.shape)
  df.dtypes
  manufacturer_part_id = -1
  for index, row in df.iterrows():
    if row["manufacturer_part_id"] == manufacturer_part_id:
      df.at[index, "start_qty"] = next_start_qty
      df.at[index, "start_qty2"] = next_start_qty2
    else:
      manufacturer_part_id = row["manufacturer_part_id"]
      next_start_qty = row["start_qty"]
      next_start_qty2 = row["start_qty2"]

    if (row["actual_is_oos"] == 1) & (row["order_qty"] == 0):
      # oos and no order at all, use lost_sales to bleed down
      next_start_qty = next_start_qty + row["other_receipt_qty"] + row["hypothetical_qty"] - row["lost_sales"]
      next_start_qty2 = next_start_qty2 + row["other_receipt_qty"] + row["hypothetical_qty2"] - row["lost_sales"]
    else:
      next_start_qty = next_start_qty + row["other_receipt_qty"] + row["hypothetical_qty"] - row["order_qty"]
      next_start_qty2 = next_start_qty2 + row["other_receipt_qty"] + row["hypothetical_qty2"] - row["order_qty"]

    # available qty can't be negative
    next_start_qty = 0 if next_start_qty < 0 else next_start_qty
    next_start_qty2 = 0 if next_start_qty2 < 0 else next_start_qty2

  df2 = df.drop(columns=["start_qty", "hypothetical_qty"])
  df2.rename(columns={"start_qty2": "start_qty", "hypothetical_qty2": "hypothetical_qty"}, inplace=True)
  df2["analysis_case"] = "hypothetical2"
  df.drop(columns=["start_qty2", "hypothetical_qty2"], inplace=True)
  df["analysis_case"] = "hypothetical1"
  df = pd.concat([df, df2])

  df["is_oos"] = np.where((df.start_qty + df.other_receipt_qty + df.hypothetical_qty - df.order_qty) <= 0, 1, 0)

  now = datetime.datetime.now()
  now = now.strftime("%Y%m%d%H%M%S")
  tmp_table_name = "tmp_lost_sales_bleeddown_" + now

  df.to_gbq(destination_table=dataset+"."+tmp_table_name, project_id=project_id, if_exists="replace")
  fq_tmp_table_name = "`" + project_id + "." + dataset + "." + tmp_table_name + "`"

  sql ="""\
      INSERT INTO `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      (
          analysis_case,
          proposal_id,
          supplier_id,
          manufacturer_part_id,
          sprid,
          date,
          is_oos,
          available_qty,
          receipt_qty,
          demand_qty,
          potential_lost_sales,
          wholesale_cost_per_unit,
          revenue_per_unit,
          expected_lost_sales,
          expected_lost_sales_by_cost,
          expected_lost_sales_by_revenue
      ) 
      SELECT
          analysis_case,
          bd.proposal_id,
          bd.supplier_id,
          bd.manufacturer_part_id,
          bd.sprid,
          EXTRACT(DATE FROM bd.date),
          bd.is_oos,
          bd.start_qty,
          bd.hypothetical_qty + bd.other_receipt_qty,
          CASE 
            WHEN bd.actual_is_oos = 1 AND bd.order_qty = 0 THEN bd.lost_sales 
            ELSE bd.order_qty END,
          bd.lost_sales,
          bd.wholesale_cost_per_unit,
          bd.revenue_per_unit,
          bd.is_oos * bd.lost_sales,
          bd.is_oos * bd.lost_sales * bd.wholesale_cost_per_unit,
          bd.is_oos * bd.lost_sales * bd.revenue_per_unit
      FROM 
          {} bd;
  """
  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  sql ="""\
    DROP TABLE IF EXISTS {}.{};
  """
  query=sql.format(dataset, tmp_table_name)
  df = client.query(query).to_dataframe()

  ###### Model Case #####
  sql ="""\
      WITH demand_forecast AS (
        SELECT
          manufacturer_part_id,
          EXTRACT(MONTH FROM start_date) AS df_month,
          EXTRACT(YEAR FROM end_date) AS df_year,
          SUM(forecast_qty_daily) AS df_qty
        FROM
          `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_demand_forecast`
        WHERE
          snapshot_id = {}
        GROUP BY
          1,2,3),
      onhand as (
        SELECT
          supplier_id,
          proposal_id,
          sprid,
          manufacturer_part_id,
          MIN(on_hand_qty) AS on_hand,
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1,2,3,4),
      proposal as (
        SELECT
          manufacturer_part_id,
          arrival_date AS date,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1,2
      ),
      inbound as (
        SELECT
          manufacturer_part_id,
          estimated_pickable_date,
          SUM(quantity) inbound_qty,
        FROM
          `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_inbound_shipment`
        WHERE snapshot_id = {}
        GROUP BY
          1,2
      ),
      date_range as (
        SELECT * FROM UNNEST(GENERATE_DATE_ARRAY('{}', '{}', INTERVAL 1 DAY)) AS date
      )
      SELECT
        onhand.proposal_id,
        onhand.supplier_id,
        onhand.manufacturer_part_id,
        onhand.sprid,
        date_range.date,
        CASE WHEN date_range.date = '{}' THEN on_hand ELSE 0 END AS start_qty,
        IFNULL(inbound.inbound_qty, 0) AS inbound_qty,
        IFNULL(quantity, 0) AS proposal_qty,
        df.df_qty AS forecast_qty,
      FROM
        onhand
      CROSS JOIN
        date_range
      LEFT JOIN
        demand_forecast AS df
      ON
        onhand.manufacturer_part_id = df.manufacturer_part_id
        AND EXTRACT(MONTH FROM date_range.date) = df.df_month
        AND EXTRACT(YEAR FROM date_range.date) = df.df_year
      LEFT JOIN 
        proposal 
      ON
        proposal.manufacturer_part_id = onhand.manufacturer_part_id 
        AND proposal.date = date_range.date
      LEFT JOIN 
        inbound 
      ON
        inbound.manufacturer_part_id = onhand.manufacturer_part_id 
        AND inbound.estimated_pickable_date = date_range.date
      ORDER BY
        onhand.manufacturer_part_id ASC,
        date_range.date ASC;
  """
  query=sql.format(snapshot_id, proposal_id, proposal_id, snapshot_id, start_date, end_date, start_date)
  df = client.query(query).to_dataframe()

  df["start_qty"] = df["start_qty"].astype(float)
  df["inbound_qty"] = df["inbound_qty"].astype(float)
  df["proposal_qty"] = df["proposal_qty"].astype(float)
  df["forecast_qty"] = df["forecast_qty"].astype(float)
  df["date"] = pd.to_datetime(df["date"])

  print (df.shape)
  df.dtypes

  manufacturer_part_id = -1
  for index, row in df.iterrows():
    if row["manufacturer_part_id"] == manufacturer_part_id:
      df.at[index, "start_qty"] = next_start_qty
    else:
      manufacturer_part_id = row["manufacturer_part_id"]
      next_start_qty = row["start_qty"]
    next_start_qty = next_start_qty + row["inbound_qty"] + row["proposal_qty"] - row["forecast_qty"]
    next_start_qty = 0 if next_start_qty < 0 else next_start_qty

  df["is_oos"] = np.where((df.start_qty + df.inbound_qty + df.proposal_qty - df.forecast_qty) <= 0, 1, 0)
  now = datetime.datetime.now()
  now = now.strftime("%Y%m%d%H%M%S")
  tmp_table_name = "tmp_lost_sales_bleeddown_" + now
  df.to_gbq(destination_table=dataset+"."+tmp_table_name, project_id=project_id, if_exists="replace")
  fq_tmp_table_name = "`" + project_id + "." + dataset + "." + tmp_table_name + "`"

  sql ="""\
      INSERT INTO `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      (
          analysis_case,
          proposal_id,
          supplier_id,
          manufacturer_part_id,
          sprid,
          date,
          is_oos,
          available_qty,
          receipt_qty,
          demand_qty,
          potential_lost_sales,
          wholesale_cost_per_unit,
          revenue_per_unit,
          expected_lost_sales,
          expected_lost_sales_by_cost,
          expected_lost_sales_by_revenue
      ) 
      SELECT
          'model',
          bd.proposal_id,
          bd.supplier_id,
          bd.manufacturer_part_id,
          bd.sprid,
          EXTRACT(DATE FROM bd.date),
          bd.is_oos,
          bd.start_qty,
          bd.proposal_qty + bd.inbound_qty,
          bd.forecast_qty,
          ls_cg.lost_sales,
          ls.wholesale_cost_per_unit,
          ls.revenue_per_unit,
          bd.is_oos * ls_cg.lost_sales,
          bd.is_oos * ls_cg.lost_sales * ls.wholesale_cost_per_unit,
          bd.is_oos * ls_cg.lost_sales * ls.revenue_per_unit
      FROM 
          {} bd
          LEFT JOIN `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_cg_only` ls_cg
            ON 
                ls_cg.date = EXTRACT(DATE FROM bd.date)
                AND ls_cg.manufacturer_part_id = bd.manufacturer_part_id
          LEFT JOIN 
            `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales` AS ls
          ON 
            ls.order_date = EXTRACT(DATE FROM bd.date)
            AND ls.manufacturer_part_id = bd.manufacturer_part_id          
      ;
  """
  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  sql ="""\
    DROP TABLE IF EXISTS {}.{};
  """
  query=sql.format(dataset, tmp_table_name)
  df = client.query(query).to_dataframe()

In [4]:
begin_time = datetime.datetime.now()

In [5]:
project_id = "wf-gcp-us-ae-buyfair-prod"
dataset = "analytics_lost_sales"

In [6]:
client = bigquery.Client(project=project_id)  
sql ="""\
    SELECT *
    FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_used`
"""
query=sql.format()
df = client.query(query).to_dataframe()
proposals = df.proposal_id.tolist()

In [7]:
# change the proposal id and run the script by Ctrl+F10
# proposals = [5862734, 5894452]

In [ ]:
for proposal_id in proposals:
  prototype_run(proposal_id)

-----5830531-----
(7905, 15)


1it [00:06,  6.61s/it]


(7905, 9)


1it [00:12, 13.00s/it]


-----5907224-----
(276, 15)


...

1it [00:05,  5.50s/it]


(8502, 9)


1it [00:05,  5.07s/it]


-----5907490-----
(6732, 15)


1it [00:00,  4.66it/s]

In [ ]:
print(datetime.datetime.now() - begin_time)